# Result

Yet another `Result` monad implementation, yet with some unconventional features.

## Table of contents

1. [Wrapping](#wrapping)
2. [Unwrapping](#unwrapping)
3. [Monadic staff](#monadic)
4. [Combining](#combining)

In [ ]:
#r "nuget: mazharenko.result, 1.0.0"
#r "nuget: mazharenko.result.Interactive, 1.0.0-beta"

using mazharenko.result;

Installed Packages mazharenko.result, 1.0.0 mazharenko.result.Interactive, 1.0.0-beta

Loading extensions from `result.Interactive.dll`

## Wrapping <a name="wrapping"></a>

There is a variety of ways to instantiate both successful and failed results depending on the context. 

In [ ]:
Result<int, string>.Success(42).Display();
Result<int, string>.Failure("something went wrong").Display();

42

something went wrong

In [ ]:
Result.From(true, 42, "something went wrong").Display();
Result.From(false, 42, "something went wrong").Display();

42

something went wrong

The most used way though is to rely on implicit cast operators. When a method is declared to return a specific `Result<,>` type and the type arguments are different, the developer can return these types instead

In [ ]:
Result<int, string> Method(bool success) {
	return success ? 42 : "something went wrong";
}

Method(true).Display();
Method(false).Display();

42

something went wrong

When dealing with `Result`s defining the same type for fail and success and for those who enjoy more explicitness, there is a couple of intermediary types which clearly say what the operation state is, but still allow not to write full generic type definitions.

In [ ]:
Result<int, string> Method(bool success) {
	return success ? Result.Success(42) : Result.Failure("something went wrong");
}

Method(true)

42

In [ ]:
Method(false)

something went wrong

## Unwrapping <a name="unwrapping"></a>

The most basic way to process data stored in the `Result` container is the `Match` method. It forces to specify how we want to process a failure. There are overloads accepting values, actions, functions, tasks.

In [ ]:
Result<int, string> success = 42;
Result<int, string> error = "something went wrong";

success.DisplayPipe()
	.Match(value => $"value {value}", er => "no value")

42

value 42

In [ ]:

error.DisplayPipe()
	.Match(value => $"value {value}", er => "no value")

something went wrong

no value

There is a set of shortcut methods `Or`, handy when the success outcome type should not change.

In [ ]:
Result<int, string>.Failure("error").DisplayPipe()
	.Or(-1)

error

-1

The imperative way to extract values from a `Result`, which can be handy in some circumstances, is using a set of `TryGet` methods. Compiler then provides *some* assistance on when one can use which variables, but the approach has some limitations, and thus is not recommended.

In [ ]:
Result<string, string> result = Result.Success("42");

result.Display();

if (result.TryGet(out var value, out var failure))
{
	value.Display();
	//failure.Display(); // would produce a warning at compile time and <null> at runtime
}
else
{
	//value.Display(); // would produce a warning at compile time and <null> at runtime
	failure.Display();
}

42

42

A collection of `Result`s can be partitioned (`Partition`) in two by the stored state or filtered with extraction (`ChooseSuccess`). For the latter failures must not be forgotten, an action to process them must be provided.

In [ ]:
new [] {
	Method(true),
	Method(false)
}.Partition()

Item1,Item2
[ 42 ],[ something went wrong ]


In [ ]:
new [] {
	Method(true),
	Method(false)
}.ChooseSuccess(f => f.Display())

something went wrong

index,value
0,42


## Monadic stuff <a name="monadic"></a>

`Result` is a functor, so it provides `Map` methods to safely map a value with a function, producing another value, possibly of another type, for a new `Result`. Also, there are methods `OrMap` to map a failure with a function.

In [ ]:
public static class IntExtensions
{
	public static Result<int, string> Parse(string from)
	{
		if (int.TryParse(from, out var value))
			return value;
		return "wrong format";
	}
}

IntExtensions.Parse("-5")
.DisplayPipe()
.Map(value => value * 100)

-5

-500

In [ ]:
IntExtensions.Parse("qwerty")
	.DisplayPipe()
	.Map(value => value * 100)

wrong format

wrong format

In [ ]:
using System.Net;

IntExtensions.Parse("-5")
	.DisplayPipe()
	.OrMap(value => HttpStatusCode.BadRequest)

-5

-5

In [ ]:
IntExtensions.Parse("qwerty")
	.DisplayPipe()
	.OrMap(value => HttpStatusCode.BadRequest)

wrong format

BadRequest

`Result` is also a monad, so it provides `Bind` methods to safely map a value with a function, producing another `Result`, possibly with another value type. Also, there are methods `OrMap` to map a failure with a function.

In [ ]:
public Result<int, string> Divide(int numerator, int denominator)
{
	return denominator switch
	{
		0 => "division by zero",
		var notZero => numerator / notZero
	};
}

IntExtensions.Parse("-5")
	.DisplayPipe()
	.Bind(value => Divide(200, value))


-5

-40

In [ ]:
IntExtensions.Parse("qwerty")
.DisplayPipe()
.Bind(value => Divide(200, value))

wrong format

wrong format

In [ ]:
IntExtensions.Parse("0")
.DisplayPipe()
.Bind(value => Divide(200, value))

0

division by zero

The same and more is achievable using LINQ do-notation

In [ ]:
from value1 in IntExtensions.Parse("qwerty").DisplayPipe()
from value2 in IntExtensions.Parse("320").DisplayPipe()
from factor in Divide(value1, value2).DisplayPipe()
select factor

wrong format

wrong format

In [ ]:
from value1 in IntExtensions.Parse("35").DisplayPipe()
from value2 in IntExtensions.Parse("0").DisplayPipe()
from factor in Divide(value1, value2)
select factor

35

0

division by zero

In [ ]:
from value1 in IntExtensions.Parse("35").DisplayPipe()
from value2 in IntExtensions.Parse("10").DisplayPipe()
from factor in Divide(value1, value2)
select factor

35

10

3

## Combining <a name="combining"></a>

### Error tree

When a certain operation depends on two other, each of which can produce failed results, we might want to return the combination of them. Here comes the concept of `Errors`. 

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab,1.0.0-beta.22314.1"

Installed Packages Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22314.1

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [ ]:
graph BT;
	B[Method 1]-->A[Root];
	C[Method 2]:::success-->B;
	D[Method 3]:::failure-->B;
	E[Method 4]:::failure-->B;
	F[Method 5]:::failure-->A
	classDef failure stroke:red,stroke-width:4px
	classDef success stroke:green,stroke-width:4px

In the example above each of the underlying methods can return a `Failure`. And the calling method might want to propagate them. The `Errors` concept says that like a program module is structured as a tree, an error produced by it can also be represented as a tree.

In [ ]:
graph BT;
	B[Failure 3+4]-->A[Root failure];
	D[Failure 3]-->B;
	E[Failure 4]-->B;
	F[Failure 5]-->A;	
	G[Failure 6]-->F
	classDef default stroke:red,stroke-width:4px

The library provides a set of types and methods to build such trees. One limitation is worth mentioning, is that it can only be built on the same type instances, so it is not compatible with the approach when each operation defines its type/enum for errors.


In [ ]:
Errors.Create("root",
	Errors.Create("3+4",
		Errors.Create("3"),
		Errors.Create("4")
	),
	Errors.Create("5",	
		Errors.Create("6")
	)
).ToString()

root
  └─> 3+4
    └─> 3
    └─> 4
  └─> 5
    └─> 6

Or, using an alternative syntax:

In [ ]:
"root".ToErrors(
	"3+4".ToErrors(
		"3".ToErrors(),
		"4".ToErrors()
	),
	"5".ToErrors(
		"6".ToErrors()
	)
).ToString()

root
  └─> 3+4
    └─> 3
    └─> 4
  └─> 5
    └─> 6

### Zip

The concept implies that there are failures in each node, not just in its leaves. When combining two failures from underlying methods, it can be useful to enrich the errors with some context from the current method.

Let's design a program that converts temperature from Fahrenheit to Kelvin. 

In [ ]:
graph BT;
	B[FarenheitToCelsius]-->A[FahrenheitToKelvin];
	C[GetRatio]-->B;
	D[GetOffset]-->B;
	F[GetKelvinOffset]-->A

In [ ]:
Result<double, string> ratio = 5d/9;
Result<double, string> GetRatio() => ratio;
Result<double, string> offset = -32;
Result<double, string> GetOffset() => offset;
Result<double, string> kelvinOffset = 273.15;
Result<double, string> GetKelvinOffset() => kelvinOffset;

In `FarenheitToCelsius` and `FahrenheitToKelvin` we want to let both errors propagate, but also enrich them with new roots.

In [ ]:
Result<double, Errors<string>> FarenheitToCelsius(double farenheit) 
{
	var ratioResult = GetRatio().ToErrors();
	var offsetResult = GetOffset().ToErrors();
	return (ratioResult, offsetResult).Zip(_ => $"Could not convert {farenheit}°F to °C") // they can be combined then with the Zip method
		.Map(t => {
			var (ratio, offset) = t;
			return (farenheit + offset) * ratio;
		});
}

Result<double, Errors<string>> FarenheitToKelvin(double farenheit) 
{
	var celciusResult = FarenheitToCelsius(farenheit);
	var kelvinOffsetResult = GetKelvinOffset().ToErrors();
	return (celciusResult, kelvinOffsetResult).Zip(_ => $"Could not convert {farenheit}°F to °K")
		.Map(t => {
			var (celcius, offset) = t;
			return celcius + offset;
		});
}

FarenheitToKelvin(32)

273.15

In [ ]:
ratio = "couldn't get ratio";

FarenheitToKelvin(32)

Could not convert 32°F to °K Could not convert 32°F to °C couldn't get ratio

In [ ]:
offset = "couldn't get offset";
kelvinOffset = "couldn't get kelvin offset";

FarenheitToKelvin(32)

Could not convert 32°F to °K Could not convert 32°F to °C couldn't get ratio couldn't get offset couldn't get kelvin offset

An enumerable of of one-type results can also be zipped, as well as individual results.

In [ ]:
new [] {
	ratio,
	offset
}.Select(ZipExtensions.ToErrors)
.Zip(failures => $"Couldn't get {failures.Count} constants")
.Zip(_ => "Couldn't convert")

Couldn't convert Couldn't get 2 constants couldn't get ratio couldn't get offset